# Hessian Matrix
### This tutorial covers the common uses for the hessian in geometry optimizations: hessian transformations, hessian guessing, and hessian updating

We will start with how to get a hessian matrix from Psi4 to be used in your optimizations. Depending on the optimization, you can calulate the hessian analytically either at the first point or at every point in the optimization, or guess and update the hessian at each point. It is also possible to caluclate the Hessian every other step or every few steps while guessing and updating in between.
We will start with calculating the Hessian for water. 

In [2]:
import psi4
import numpy as np
from optking import printTools
printTools.printInit(printTools.cleanPrint)
from optking import molsys
from optking import intcosMisc
from optking import addIntcos
from optking import optParams as op
mol = psi4.geometry("""
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

# Set some options
psi4.set_options({"basis": "cc-pvdz",
                  "scf_type": "pk",
                  "e_convergence": 1e-8})

H = psi4.driver.hessian('scf')
H = np.array(H)
printTools.print_opt("3N * 3N Hessian Matrix\n")
printTools.printMat(H)

3N * 3N Hessian Matrix
   0.097366   0.000000  -0.000000  -0.048683  -0.000000   0.000000  -0.048683
   0.000000   0.000000
   0.000000   0.290842   0.000000   0.000000  -0.145421   0.075580  -0.000000
  -0.145421  -0.075580
  -0.000000   0.000000   0.279184   0.000000   0.030890  -0.139592   0.000000
  -0.030890  -0.139592
  -0.048683   0.000000   0.000000   0.051456   0.000000  -0.000000  -0.002773
  -0.000000   0.000000
  -0.000000  -0.145421   0.030890   0.000000   0.163592  -0.053235   0.000000
  -0.018171   0.022345
   0.000000   0.075580  -0.139592  -0.000000  -0.053235   0.126435   0.000000
  -0.022345   0.013157
  -0.048683  -0.000000   0.000000  -0.002773   0.000000   0.000000   0.051456
  -0.000000  -0.000000
   0.000000  -0.145421  -0.030890  -0.000000  -0.018171  -0.022345  -0.000000
   0.163592   0.053235
   0.000000  -0.075580  -0.139592   0.000000   0.022345   0.013157  -0.000000
   0.053235   0.126435


In order to guess the Hessian, we need to create the internal coordiantes for the molecules and the molecular system. Each internal coordinate has a method then to guess its own force constant using a variety of published methods, here we use the default 'SIMPLE' method. This next cell will take some time to run because of the work required to get the keyowrds from psi4

In [3]:
mol.update_geometry()
xyz = np.array(mol.geometry())
 
from optking import stre, bend, tors

s1 = stre.STRE(0,1)
s2 = stre.STRE(0,2)
theta = bend.BEND(1,0,2)

intcos = (s1, s2, theta)
for intco in intcos: 
    print (intco)
    
#This creates a moecualr system which can be one or more molecules

Molsys = molsys.MOLSYS.fromPsi4Molecule(mol)

#This gives optking the keywords and options it needs from Psi4
userOptions = psi4.driver.p4util.prepare_options_for_modules()
op.Params = op.OPT_PARAMS(userOptions)

#grabs the molecular system's list of atomic numbers for its atoms
Z = Molsys.Z
connectivity = addIntcos.connectivityFromDistances(xyz, Z)

hessianValues = [] 
for intco in intcos:
    hessianValues.append(intco.diagonalHessianGuess(xyz, Z, connectivity))

print("\nForce constants for each internal coordinate")    
print(hessianValues)

 R(1,2)
 R(1,3)
 B(2,1,3)

	Generating molecular system for optimization from PSI4.
	1 Fragments in PSI4 molecule object.
	Creating fragment 1 with 3 atoms

Force constants for each internal coordinate
[0.5, 0.5, 0.2]


Using these values we can create our hessian

In [7]:
guessHessian = np.zeros((3, 3), float)
for i in range (3):
    guessHessian[i][i] = hessianValues[i]
print (guessHessian)

[[ 0.5  0.   0. ]
 [ 0.   0.5  0. ]
 [ 0.   0.   0.2]]


As previously seen, Psi4 calculates the hessian in cartesians; however, optking guesses and updates the Hessian in internals. TO show how to convert the hessian between coordinates, let's start at the definition of the hessian.
$$ \textbf H = \frac{\partial ^2 E}{\partial x_i\partial x_j}$$
substituting in the partial deriviative of internal coordinates
$$ \textbf H = \frac{\partial}{\partial x_i}\Big( \frac{\partial E}{\partial q} \frac{\partial q}{\partial x_j}\Big )$$
$$ = \frac{\partial ^2 E}{\partial x_i \partial q} \frac{\partial q}{\partial x_j} + \frac{\partial E}{\partial q}\frac{\partial ^2 q}{\partial x_i \partial x_j}$$
Adding the partial derivative of internals to the first term
$$ \frac{\partial ^2 E}{\partial q \partial q} \frac{ \partial q \partial q}{\partial x_i \partial x_j} + \frac{\partial E }{\partial q} \frac {\partial ^2 q}{\partial x_i \partial x_j}$$
This can now be reduced to matrix form, rearranging to match the dimensions of the matrices.
$$ \textbf H = \textbf B^T \textbf {H B} + g \textbf B_{ij}$$
At stationary points, the second term can be dropped allowing for the calculation of the second derivative B matrix to be skipped

In order to conver the Hessian into internals, which is more commonly used in calculations, we use the generalized inverse of $\textbf B^T$, the A matrix $\big ( \textbf B^T u \textbf B \big) ^{-1} \textbf B u$

$$\textbf A^T \textbf H_x \textbf A = \textbf A^T  \textbf B^T \textbf H_q \textbf {BA} + \textbf A^Tg \textbf B_{ij} \textbf A$$

$$\textbf A^T \textbf H_x \textbf A - \textbf A^T g \textbf B_{ij} \textbf A = \textbf H$$

Now lets convert the Psi4 hessian into internals

In [8]:
masses = Molsys.masses
G = intcosMisc.Gmat(intcos, xyz, masses)
Ginv = symmMatInv(G)
B = intcosMisc.Bmat(intcos, xyz, masses)
Atranspose = np.dot(Ginv, B)

if g_x is None:  # A^t Hxy A
    print_opt("Neglecting force/B-matrix derivative term, only correct at stationary points.\n")
    Hworking = H
else: # A^t (Hxy - Kxy) A;    K_xy = sum_q ( grad_q[I] d^2(q_I)/(dx dy) )
    print_opt("Including force/B-matrix derivative term.\n")
    Hworking = H.copy()

    g_q = np.dot(Atranspose, g_x)
    Ncart = 3*len(geom)
    dq2dx2 = np.zeros((Ncart,Ncart), float)  # should be cart x cart for fragment ?

    for I, q in enumerate(intcos):
        dq2dx2[:] = 0
        q.Dq2Dx2(geom, dq2dx2)   # d^2(q_I)/ dx_i dx_j

        for a in range(Ncart):
            for b in range(Ncart):
                Hworking[a,b] -= g_q[I] * dq2dx2[a,b] # adjust indices for multiple fragments

Hq = np.dot(Atranspose, np.dot(Hworking, Atranspose.T))
print (Hq)

TypeError: 'NoneType' object is not callable

And to convert back

In [19]:
B = intcosMisc.Bmat(intcos, geom, masses)
Hxy = np.dot(B.T, np.dot(Hint, B))

if g_q is None:  # Hxy =  B^t Hij B
    print_opt("Neglecting force/B-matrix derivative term, only correct at stationary points.\n")
else:            # Hxy += dE/dq_I d2(q_I)/dxdy
    print_opt("Including force/B-matrix derivative term.\n")
    Ncart = 3 * len(geom)

    dq2dx2 = np.zeros((Ncart, Ncart), float)  # should be cart x cart for fragment ?
    for I, q in enumerate(intcos):
        dq2dx2[:] = 0
        q.Dq2Dx2(geom, dq2dx2);

        for a in range(Ncart):
            for b in range(Ncart):
                Hxy[a, b] += g_q[I] * dq2dx2[a,b]
                    
print (Hxy)                    

NameError: name 'geom' is not defined

In [9]:
        R = v3d.dist(geom[self.A],geom[self.B])
        PerA = ZtoPeriod(Z[self.A])
        PerB = ZtoPeriod(Z[self.B])

        AA = 1.734
        if PerA == 1:
            if PerB == 1:
                BB = -0.244
            elif PerB == 2:
            BB = 0.352
                else:
            BB = 0.660
        elif PerA == 2:
            if PerB == 1:
            BB = 0.352
                elif PerB == 2:
            BB = 1.085
            else:
            BB = 1.522
        else:
            if PerB == 1:
            BB = 0.660
                elif PerB == 2:
            BB = 1.522
                else:
            BB = 2.068
            
        F = AA/((R-BB)*(R-BB)*(R-BB))
        return F
                                                                                                      156,0-1       71%

IndentationError: expected an indented block (<ipython-input-9-96fdb1679891>, line 10)